***
10 random seeds: range(20, 30)
for data creation for each type of spammer

invoke factor bt

get factor bt accuracy (+- std dev), wacc and tau

save in results/spammer_type/factor_bt.csv
***

## Device Setup

In [1]:
!nvidia-smi

Sun Dec 21 08:14:40 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   50C    P0              67W / 300W |    529MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
print(f"Current PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

Current PyTorch version: 2.9.1+cu128
CUDA available: True
CUDA version: 12.8


### Importing FactorBT

In [4]:
import sys
sys.path.insert(0, "../")
sys.path.insert(1, "../../")

from spammer_types import *
from util import *
import opt_fair
from crowdkit.aggregation import NoisyBradleyTerry
from distribution_utils import crowd_bt_dist, logistic_preference_dist, comparisons_to_df, safe_kendalltau, to_numpy
from metrics import compute_acc, compute_weighted_acc
from pgem import EMWrapper

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Passage dataset

### Get the original df of passage dataset

In [5]:
df_path = "../../real_data/passage/data/passage_cleaned.csv"

In [6]:
import pandas as pd
df = pd.read_csv(df_path)
def sort_df(df, column_name):
        # Sort by a specific column (replace 'column_name' with your column)
        df_sorted = df.sort_values(by=column_name, ascending=True)  # or ascending=False

        return df_sorted
df = sort_df(df, 'performer')
df[['left', 'right', 'label', 'performer']].head()

,left,right,label,performer
0,been wicked. They believed that the end of the...,lichen Sect Content Linking Artid A snake coil...,lichen Sect Content Linking Artid A snake coil...,5
20,"school, you noticed that all the clocks read a...",man in the seat with me and the two women acro...,man in the seat with me and the two women acro...,5
19,foreign animals or plants may be taken into th...,adhere to the wall. Using sizing that has been...,foreign animals or plants may be taken into th...,5
18,many pennies did each child get Which computat...,the picture on its cover. Write two or three s...,many pennies did each child get Which computat...,5
17,direction of the Sun. The length in meters and...,"the ground going into the cave. Oh, my he said...","the ground going into the cave. Oh, my he said...",5


In [7]:
gt_df_path = "../../real_data/passage/data/gt_df_passage.csv"

In [8]:
import pandas as pd
gt_df = pd.read_csv(gt_df_path)
def sort_df(df, column_name):
        # Sort by a specific column (replace 'column_name' with your column)
        df_sorted = df.sort_values(by=column_name, ascending=True)  # or ascending=False

        return df_sorted
gt_df = sort_df(gt_df, 'label')
gt_df.head()

,Unnamed: 0,label,score
333,119,'Arc Vallon Pont 'Arc One interesting thing yo...,6
244,109,. . Main engine ignition sequence started . . ...,4
436,432,. and to explore the outdoor world. The packs ...,11
432,22,". early invading species like horseweed, aster...",11
321,361,Accords. The Accords were approved in May by p...,6


In [9]:
percents = [10, 20, 40, 60, 80]
# percents = [10]

In [10]:
import pickle

with open("../../real_data/passage/data/PassageDF.pickle", "rb") as handle:
    df_passage = pickle.load(handle)
df_passage

,label,score
0,"a star. Our planet, Earth, orbits, or circles,...",1
1,"Adam, We did not have plastic toys. I played w...",1
2,Who said the little owl. Who wants to hunt wit...,1
3,dead leaf. This is a mole. Moles burrow underg...,1
4,ereaddatagradepsenvironcomp.html Environment r...,1
...,...,...
467,work over the summer on any changes they wish ...,12
468,between January and December plunged the Unite...,12
469,into a newly opened bank account. I was amazed...,12
470,"occurring phenomenon, manmade by products are ...",12


In [11]:
size = len(df_passage)
print(size)
classes = [0] * size
# for faceage it would be classes = df_passage['gender']

472


In [12]:
def sort_df(df, column_name):
    # Sort by a specific column (replace 'column_name' with your column)
    df_sorted = df.sort_values(by=column_name, ascending=True)  # or ascending=False

    return df_sorted

### Addition of Random Guessors

In [13]:
spammer_type = "random"

In [14]:
csv_file = f"results/{spammer_type}/factorbt.csv"

In [15]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [16]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "FactorBT_acc_mean", "FactorBT_acc_std",
    "FactorBT_wacc_mean", "FactorBT_wacc_std",
    "FactorBT_tau_mean", "FactorBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [17]:
max_iter_pgem = 100
import pandas as pd

for percent in percents:
    # initialize metrics
    FactorBT_accs, FactorBT_waccs, FactorBT_taus = [], [], []
    
    for sd in range(20, 30):
        try:
            # get df
            random_df, spammer_ids = add_random_spammer(df, percent, seed=sd)
            K = len(list(random_df["performer"].unique()))
            print(K)
            random_df.rename(columns={"performer": "worker"}, inplace=True)
        
            agg_noisybt = NoisyBradleyTerry(n_iter=10).fit_predict(random_df)
            agg_noisybt_df = pd.DataFrame(list(agg_noisybt.items()), columns=['label', 'score'])
            factorbt_scores = list(agg_noisybt_df['score'])
            annot_bt_np = to_numpy(factorbt_scores)
            
            if np.isnan(annot_bt_np).any():
                    continue
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if FactorBT_tau < 0:
                annot_bt_np = -annot_bt_np
            FactorBT_acc = compute_acc(gt_df, annot_bt_np, device)
            FactorBT_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"FactorBT failed due to {e}")
            continue
        FactorBT_accs.append(FactorBT_acc)
        FactorBT_waccs.append(FactorBT_wacc)
        FactorBT_taus.append(FactorBT_tau)

    row = [
        percent,
        np.mean(FactorBT_accs), np.std(FactorBT_accs),
        np.mean(FactorBT_waccs), np.std(FactorBT_waccs),
        np.mean(FactorBT_taus), np.std(FactorBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"FactorBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(FactorBT_accs):.4f} ± {np.std(FactorBT_accs):.4f} | "
    f"WAcc: {np.mean(FactorBT_waccs):.4f} ± {np.std(FactorBT_waccs):.4f} | "
    f"Tau: {np.mean(FactorBT_taus):.4f} ± {np.std(FactorBT_taus):.4f}")

686
686
686
686
686
686
686
686
686
686
FactorBT | Percent: 10 |Acc: 0.6968 ± 0.0021 | WAcc: 0.7436 ± 0.0037 | Tau: 0.3716 ± 0.0040
748
748
748
748
748
748
748
748
748
748
FactorBT | Percent: 20 |Acc: 0.6935 ± 0.0032 | WAcc: 0.7431 ± 0.0036 | Tau: 0.3652 ± 0.0060
873
873
873
873
873
873
873
873
873
873
FactorBT | Percent: 40 |Acc: 0.6940 ± 0.0022 | WAcc: 0.7409 ± 0.0045 | Tau: 0.3663 ± 0.0041
998
998
998
998
998
998
998
998
998
998
FactorBT | Percent: 60 |Acc: 0.6919 ± 0.0032 | WAcc: 0.7402 ± 0.0046 | Tau: 0.3624 ± 0.0060
1123
1123
1123
1123
1123
1123
1123
1123
1123
1123
FactorBT | Percent: 80 |Acc: 0.6867 ± 0.0046 | WAcc: 0.7364 ± 0.0068 | Tau: 0.3526 ± 0.0087


### Addition of Anti-Personas

In [18]:
spammer_type = "anti"

In [19]:
csv_file = f"results/{spammer_type}/factorbt.csv"

In [20]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [21]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "FactorBT_acc_mean", "FactorBT_acc_std",
    "FactorBT_wacc_mean", "FactorBT_wacc_std",
    "FactorBT_tau_mean", "FactorBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [22]:
max_iter_pgem = 100
import pandas as pd

for percent in percents:
    # initialize metrics
    FactorBT_accs, FactorBT_waccs, FactorBT_taus = [], [], []
    
    for sd in range(20, 30):
        try:
            # get df
            random_df, spammer_ids = add_anti_personas(df, percent, seed=sd)
            K = len(list(random_df["performer"].unique()))
            print(K)
            random_df.rename(columns={"performer": "worker"}, inplace=True)
        
            agg_noisybt = NoisyBradleyTerry(n_iter=10).fit_predict(random_df)
            agg_noisybt_df = pd.DataFrame(list(agg_noisybt.items()), columns=['label', 'score'])
            factorbt_scores = list(agg_noisybt_df['score'])
            annot_bt_np = to_numpy(factorbt_scores)
            
            if np.isnan(annot_bt_np).any():
                    continue
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if FactorBT_tau < 0:
                annot_bt_np = -annot_bt_np
            FactorBT_acc = compute_acc(gt_df, annot_bt_np, device)
            FactorBT_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"FactorBT failed due to {e}")
            continue
        FactorBT_accs.append(FactorBT_acc)
        FactorBT_waccs.append(FactorBT_wacc)
        FactorBT_taus.append(FactorBT_tau)

    row = [
        percent,
        np.mean(FactorBT_accs), np.std(FactorBT_accs),
        np.mean(FactorBT_waccs), np.std(FactorBT_waccs),
        np.mean(FactorBT_taus), np.std(FactorBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"FactorBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(FactorBT_accs):.4f} ± {np.std(FactorBT_accs):.4f} | "
    f"WAcc: {np.mean(FactorBT_waccs):.4f} ± {np.std(FactorBT_waccs):.4f} | "
    f"Tau: {np.mean(FactorBT_taus):.4f} ± {np.std(FactorBT_taus):.4f}")

686
686
686
686
686
686
686
686
686
686
FactorBT | Percent: 10 |Acc: 0.6953 ± 0.0011 | WAcc: 0.7431 ± 0.0013 | Tau: 0.3688 ± 0.0020
748
748
748
748
748
748
748
748
748
748
FactorBT | Percent: 20 |Acc: 0.6910 ± 0.0015 | WAcc: 0.7370 ± 0.0014 | Tau: 0.3607 ± 0.0028
873
873
873
873
873
873
873
873
873
873
FactorBT | Percent: 40 |Acc: 0.6790 ± 0.0015 | WAcc: 0.7201 ± 0.0021 | Tau: 0.3380 ± 0.0028
998
998
998
998
998
998
998
998
998
998
FactorBT | Percent: 60 |Acc: 0.6518 ± 0.0017 | WAcc: 0.6962 ± 0.0019 | Tau: 0.2865 ± 0.0032
1123
1123
1123
1123
1123
1123
1123
1123
1123
1123
FactorBT | Percent: 80 |Acc: 0.6022 ± 0.0015 | WAcc: 0.6272 ± 0.0027 | Tau: 0.1930 ± 0.0028


### Addition of Left Position-Biased Spammers

In [23]:
spammer_type = "left"

In [24]:
csv_file = f"results/{spammer_type}/factorbt.csv"

In [25]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [26]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "FactorBT_acc_mean", "FactorBT_acc_std",
    "FactorBT_wacc_mean", "FactorBT_wacc_std",
    "FactorBT_tau_mean", "FactorBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100
import pandas as pd

for percent in percents:
    # initialize metrics
    FactorBT_accs, FactorBT_waccs, FactorBT_taus = [], [], []
    
    for sd in range(20, 30):
        try:
            # get df
            random_df, spammer_ids = add_position_biased_spammers(df, percent,position_bias="left", seed=sd)
            K = len(list(random_df["performer"].unique()))
            print(K)
            random_df.rename(columns={"performer": "worker"}, inplace=True)
        
            agg_noisybt = NoisyBradleyTerry(n_iter=10).fit_predict(random_df)
            agg_noisybt_df = pd.DataFrame(list(agg_noisybt.items()), columns=['label', 'score'])
            factorbt_scores = list(agg_noisybt_df['score'])
            annot_bt_np = to_numpy(factorbt_scores)
            
            if np.isnan(annot_bt_np).any():
                    continue
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if FactorBT_tau < 0:
                annot_bt_np = -annot_bt_np
            FactorBT_acc = compute_acc(gt_df, annot_bt_np, device)
            FactorBT_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"FactorBT failed due to {e}")
            continue
        FactorBT_accs.append(FactorBT_acc)
        FactorBT_waccs.append(FactorBT_wacc)
        FactorBT_taus.append(FactorBT_tau)

    row = [
        percent,
        np.mean(FactorBT_accs), np.std(FactorBT_accs),
        np.mean(FactorBT_waccs), np.std(FactorBT_waccs),
        np.mean(FactorBT_taus), np.std(FactorBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"FactorBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(FactorBT_accs):.4f} ± {np.std(FactorBT_accs):.4f} | "
    f"WAcc: {np.mean(FactorBT_waccs):.4f} ± {np.std(FactorBT_waccs):.4f} | "
    f"Tau: {np.mean(FactorBT_taus):.4f} ± {np.std(FactorBT_taus):.4f}")

686
686
686


### Addition of Right Position Biased Spammers

In [ ]:
spammer_type = "right"

In [ ]:
csv_file = f"results/{spammer_type}/factorbt.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "FactorBT_acc_mean", "FactorBT_acc_std",
    "FactorBT_wacc_mean", "FactorBT_wacc_std",
    "FactorBT_tau_mean", "FactorBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100
import pandas as pd

for percent in percents:
    # initialize metrics
    FactorBT_accs, FactorBT_waccs, FactorBT_taus = [], [], []
    
    for sd in range(20, 30):
        try:
            # get df
            random_df, spammer_ids = add_position_biased_spammers(df, percent,position_bias="right", seed=sd)
            K = len(list(random_df["performer"].unique()))
            print(K)
            random_df.rename(columns={"performer": "worker"}, inplace=True)
        
            agg_noisybt = NoisyBradleyTerry(n_iter=10).fit_predict(random_df)
            agg_noisybt_df = pd.DataFrame(list(agg_noisybt.items()), columns=['label', 'score'])
            factorbt_scores = list(agg_noisybt_df['score'])
            annot_bt_np = to_numpy(factorbt_scores)
            
            if np.isnan(annot_bt_np).any():
                    continue
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if FactorBT_tau < 0:
                annot_bt_np = -annot_bt_np
            FactorBT_acc = compute_acc(gt_df, annot_bt_np, device)
            FactorBT_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"FactorBT failed due to {e}")
            continue
        FactorBT_accs.append(FactorBT_acc)
        FactorBT_waccs.append(FactorBT_wacc)
        FactorBT_taus.append(FactorBT_tau)

    row = [
        percent,
        np.mean(FactorBT_accs), np.std(FactorBT_accs),
        np.mean(FactorBT_waccs), np.std(FactorBT_waccs),
        np.mean(FactorBT_taus), np.std(FactorBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"FactorBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(FactorBT_accs):.4f} ± {np.std(FactorBT_accs):.4f} | "
    f"WAcc: {np.mean(FactorBT_waccs):.4f} ± {np.std(FactorBT_waccs):.4f} | "
    f"Tau: {np.mean(FactorBT_taus):.4f} ± {np.std(FactorBT_taus):.4f}")

### Addition of Equal Proportion of all four kind of spammers

In [ ]:
spammer_type = "equal"

In [ ]:
csv_file = f"results/{spammer_type}/factorbt.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "FactorBT_acc_mean", "FactorBT_acc_std",
    "FactorBT_wacc_mean", "FactorBT_wacc_std",
    "FactorBT_tau_mean", "FactorBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100
import pandas as pd

for percent in percents:
    # initialize metrics
    FactorBT_accs, FactorBT_waccs, FactorBT_taus = [], [], []
    
    for sd in range(20, 30):
        try:
            # get df
            random_df, spammer_ids = add_equal_proportion_of_all_spammers(df, percent, seed=sd)
            K = len(list(random_df["performer"].unique()))
            print(K)
            random_df.rename(columns={"performer": "worker"}, inplace=True)
        
            agg_noisybt = NoisyBradleyTerry(n_iter=10).fit_predict(random_df)
            agg_noisybt_df = pd.DataFrame(list(agg_noisybt.items()), columns=['label', 'score'])
            factorbt_scores = list(agg_noisybt_df['score'])
            annot_bt_np = to_numpy(factorbt_scores)
            
            if np.isnan(annot_bt_np).any():
                    continue
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if FactorBT_tau < 0:
                annot_bt_np = -annot_bt_np
            FactorBT_acc = compute_acc(gt_df, annot_bt_np, device)
            FactorBT_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"FactorBT failed due to {e}")
            continue
        FactorBT_accs.append(FactorBT_acc)
        FactorBT_waccs.append(FactorBT_wacc)
        FactorBT_taus.append(FactorBT_tau)

    row = [
        percent,
        np.mean(FactorBT_accs), np.std(FactorBT_accs),
        np.mean(FactorBT_waccs), np.std(FactorBT_waccs),
        np.mean(FactorBT_taus), np.std(FactorBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"FactorBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(FactorBT_accs):.4f} ± {np.std(FactorBT_accs):.4f} | "
    f"WAcc: {np.mean(FactorBT_waccs):.4f} ± {np.std(FactorBT_waccs):.4f} | "
    f"Tau: {np.mean(FactorBT_taus):.4f} ± {np.std(FactorBT_taus):.4f}")